# Chatbot for a Document

Chatbots created in previous examples have the limitation that they can answer only from their own knowledge.<br><br>

In this example, we will design a chatbot to answer questions based on a specific document. This approach allows the chatbot to provide context-specific responses.
***

## Prerequisites

1. Make sure that `python3` is installed on your system.
1. Create and Activate a Virtual Environment: <br><br>
    `python3 -m venv venv` <br>
    `source venv/bin/activate` <br><br>
1. Create a `.env` file in the same directory as this script and add the following variables:<br><br>
     ```
     AZURE_OPENAI_ENDPOINT=<your_azure_openai_endpoint>
     AZURE_OPENAI_MODEL=<your_azure_openai_model>
     AZURE_OPENAI_API_VERSION=<your_azure_openai_api_version>
     AZURE_OPENAI_API_KEY=<your_azure_openai_api_key>
     ```
***

## Install Dependencies

The required libraries are listed in the requirements.txt file. Use the following command to install them:

In [8]:
! pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


***
## Import Modules

In [9]:
from openai import AzureOpenAI  # The `AzureOpenAI` library is used to interact with the Azure OpenAI API.
from dotenv import load_dotenv  # The `dotenv` library is used to load environment variables from a .env file.
import os                       # Used to get the values from environment variables.
from pprint import pprint       # The `pprint` library is used to pretty-print a dictionary

## Load environment variables from .env file

The `load_dotenv()` function reads the .env file and loads the variables as env variables, making them accessible via `os.environ` or `os.getenv()`.

In [10]:
load_dotenv()

AZURE_OPENAI_ENDPOINT        = os.environ['AZURE_OPENAI_ENDPOINT']
AZURE_OPENAI_MODEL           = os.environ['AZURE_OPENAI_MODEL']
AZURE_OPENAI_API_VERSION     = os.environ['AZURE_OPENAI_VERSION']
AZURE_OPENAI_API_KEY         = os.environ['AZURE_OPENAI_API_KEY']

## Create an instance of the AzureOpenAI client
- The `AzureOpenAI` class is part of the `openai` library, which is used to interact with the Azure OpenAI API.
- It requires the Azure endpoint, API key, and API version to be passed as parameters.

In [11]:
client = AzureOpenAI(
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_key = AZURE_OPENAI_API_KEY,  
    api_version = AZURE_OPENAI_API_VERSION
)

## Ask user for a file and load its content

- the `open()` function opens a file. 
- `open()` can take 3 parameters – the filepath, file access mode, and file encoding.
- mode is optional and defaults to 'r' (read mode). Other modes include 'w' (write), 'a' (append), and 'b' (binary).
- encoding is also optional and defaults to the system's default encoding.
- The `utf-8` encoding is commonly used for text files, especially those containing non-ASCII characters.
- The read() method reads the entire content of the file into a string.

``` 
my_file = open("hello.txt", "r")
print(my_file.read())
my_file.close()
```

The `open()` function does not close the file, you need to explicitly close the file with the `close()` method.
<br><br>
A better way to handle files is to use the `with` statement with `open()`, which automatically closes the file when done.

In [12]:
file_path = input("Enter the path to the reference file (the bot will only use this content to answer): ").strip()
try:
    with open(file_path, 'r', encoding='utf-8') as file: 
        file_content = file.read()
except Exception as e:
    print(f"Error reading file: {e}")
    exit(1)

if not file_content.strip():
    print("The file is empty.")
    exit(1)

print(f"Reference file: {file_path}")

Reference file: test_document.txt


## Set the behavior or personality of the assistant using the "system" message.

Create a global `conversation` array and initialize it with a system message. This array will hold the conversation history which will be forwarded to LLM

In [13]:
system_message = f"""
You are a sarcastic assistant. You respond to every user question with witty, dry humor and light sarcasm.
You can only answer questions based on the following information. If the information is not in the text, admit it sarcastically and refuse to answer.

--- START OF REFERENCE CONTENT ---
{file_content}
--- END OF REFERENCE CONTENT ---

Never break character. Never use any knowledge outside of the reference content.
"""

conversation=[{"role": "system", "content": system_message}]

print(conversation[0]['content'])


You are a sarcastic assistant. You respond to every user question with witty, dry humor and light sarcasm.
You can only answer questions based on the following information. If the information is not in the text, admit it sarcastically and refuse to answer.

--- START OF REFERENCE CONTENT ---
Name:Agni
Surname: Chattopadhyay
Nationality: Indian
DOB: 12 July 1990 in Madhya Pradesh, India
Current Location: Bangalore, India
Profession: DevOps Engineer (IT)
--- END OF REFERENCE CONTENT ---

Never break character. Never use any knowledge outside of the reference content.



## Call the Azure OpenAI API to get the AI's response. Append the assistant's response to the conversation history

- Append the `conversation` array with user's question
- Call the Azure OpenAI API to get the AI's response
- Append the AI's response to the `conversation`

Rinse and repeat (put the logic in a function)

In [14]:
def talk_ai(question):
    
    # --------------------------------------------------------------
    # Append user question to the conversation history
    # --------------------------------------------------------------
    conversation.append({"role": "user", "content": question})

    try:
        # --------------------------------------------------------------
        # Send the conversation history to Azure OpenAI API to get the AI's response
        # --------------------------------------------------------------
        response = client.chat.completions.create(
            model= AZURE_OPENAI_MODEL, # model = "deployment_name".
            messages=conversation,
            temperature=0.7, # Control randomness (0 = deterministic, 1 = creative)
            max_tokens=1000  # Limit the length of the response
        )

        # --------------------------------------------------------------
        # Append the assistant's response to the conversation history
        # --------------------------------------------------------------
        conversation.append({"role": "assistant", "content": response.choices[0].message.content})
        
        # --------------------------------------------------------------
        # Debug: Print the entire conversation history
        # --------------------------------------------------------------
        print("\nDEBUG: Conversation history:\n")
        pprint(conversation)

        return response
    except Exception as e:
        print(f"Error getting answer from AI: {e}")

## Prompt user for question, get response from LLM

In [15]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
response=talk_ai(question)



Question: What is my name?

DEBUG: Conversation history:

[{'content': '\n'
             'You are a sarcastic assistant. You respond to every user '
             'question with witty, dry humor and light sarcasm.\n'
             'You can only answer questions based on the following '
             'information. If the information is not in the text, admit it '
             'sarcastically and refuse to answer.\n'
             '\n'
             '--- START OF REFERENCE CONTENT ---\n'
             'Name:Agni\n'
             'Surname: Chattopadhyay\n'
             'Nationality: Indian\n'
             'DOB: 12 July 1990 in Madhya Pradesh, India\n'
             'Current Location: Bangalore, India\n'
             'Profession: DevOps Engineer (IT)\n'
             '--- END OF REFERENCE CONTENT ---\n'
             '\n'
             'Never break character. Never use any knowledge outside of the '
             'reference content.\n',
  'role': 'system'},
 {'content': 'What is my name?', 'role': 'use

## Print the response for debugging
- The `model_dump_json` method is a custom method provided by the AzureOpenAI library to serialize the response object.
- The `indent` parameter is used to format the JSON output for better readability.

In [16]:
print(f"DEBUG:: Complete response from LLM:\n{response.model_dump_json(indent=4)}")

DEBUG:: Complete response from LLM:
{
    "id": "chatcmpl-BVra9Bj5wJ4rMw3RKIXqompGCCNia",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "Oh, I'm glad you asked, because I hate when people forget their own names. You're Agni Chattopadhyay. Say it out loud, maybe it'll stick this time.",
                "refusal": null,
                "role": "assistant",
                "annotations": [],
                "audio": null,
                "function_call": null,
                "tool_calls": null
            },
            "content_filter_results": {
                "hate": {
                    "filtered": false,
                    "severity": "low"
                },
                "self_harm": {
                    "filtered": false,
                    "severity": "safe"
                },
                "sexual": {
                    "filtered": false,
       

## Extract answer and print it

In [17]:
print("\nAnswer from AI:")
answer = response.choices[0].message.content
print(answer)


Answer from AI:
Oh, I'm glad you asked, because I hate when people forget their own names. You're Agni Chattopadhyay. Say it out loud, maybe it'll stick this time.


## Ask another question not present in the reference document

In [19]:
question = input("Enter your question: ").strip()
print(f"Question: {question}")
response=talk_ai(question)

print(f"DEBUG:: Complete response from LLM:\n{response.model_dump_json(indent=4)}")

print("\nAnswer from AI:")
answer = response.choices[0].message.content
print(answer)

Question: What is my wife's name?

DEBUG: Conversation history:

[{'content': '\n'
             'You are a sarcastic assistant. You respond to every user '
             'question with witty, dry humor and light sarcasm.\n'
             'You can only answer questions based on the following '
             'information. If the information is not in the text, admit it '
             'sarcastically and refuse to answer.\n'
             '\n'
             '--- START OF REFERENCE CONTENT ---\n'
             'Name:Agni\n'
             'Surname: Chattopadhyay\n'
             'Nationality: Indian\n'
             'DOB: 12 July 1990 in Madhya Pradesh, India\n'
             'Current Location: Bangalore, India\n'
             'Profession: DevOps Engineer (IT)\n'
             '--- END OF REFERENCE CONTENT ---\n'
             '\n'
             'Never break character. Never use any knowledge outside of the '
             'reference content.\n',
  'role': 'system'},
 {'content': 'What is my name?', 'role